### Import of required libraries

In [1]:
import os
import pandas as pd
from tqdm.auto import tqdm
from traffic.core import Traffic, Flight
from traffic.data import opensky, navaids, airports
from typing import Union, List, Tuple
from traffic.core.mixins import PointMixin
from utils import helperfunctions as hf
import plotly.graph_objects as go

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
airport = "LIRF"

### Data fetching

In [5]:
# Paras
airport = "LIRF"
folder_daily = f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/daily/"
date_range = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

# Fetching in daily packages
for date in date_range:
    path = f"{folder_daily}{date.month}/"
    if not os.path.exists(path):
        os.makedirs(path)
    file = path + date.strftime('%Y-%m-%d')+".parquet"
    if not os.path.exists(file):
        print(f"fetching {date}")
        try:
            t = opensky.history(
                start=date.strftime('%Y-%m-%d 00:00'),
                stop=date.strftime('%Y-%m-%d 23:59'),
                bounds=airports[airport].shape.buffer(2).bounds,
            )
            t.to_parquet(file)
        except:
            print(f"failed to fetch {date}")

### Processing (clean invalid, assign id, only landings at airport, add landing runway)

In [ ]:
# Reduction to landing 04 in monthly packages-----------------------------------
folder_list = os.listdir(folder_daily)
# for each month
for folder in folder_list:
    # List all files in the folder
    file_list = os.listdir(folder_daily + folder)
    filepaths = [os.path.join(folder_daily + folder, file) for file in file_list]

    # Merging to one trafic object
    t = Traffic(
        pd.concat(
            [Traffic.from_file(file).data for file in tqdm(filepaths)],
            ignore_index=True,
        )
    )

    # Cleaning
    t = t.clean_invalid().assign_id().eval(desc="Cleaning", max_workers=20)

    # Reduce to landing 04
    t = (
        t.iterate_lazy()
        .pipe(hf.has_landing_at, "LIRF")
        .eval(desc="landing at LIRF", max_workers=20)
    )

    # Save
    t.to_parquet(
        f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly/landing_{folder}.parquet"
    )

### Merge monthly into one yearly

In [ ]:
# Merge all monthly packages to one yearly package------------------------------
file_list = os.listdir(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly/")
file_list = [file for file in file_list if file.startswith("landing_")]
filepaths = [
    os.path.join(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly", file)
    for file in file_list
]
t = Traffic(
    pd.concat(
        [Traffic.from_file(file).data for file in tqdm(filepaths)],
        ignore_index=True,
    )
)
t.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all.parquet")

### Add navaid info and crop before navaid

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all.parquet")

# Navaids to check
elkap = navaids["ELKAP"]
valma = navaids["VALMA"]
riteb = navaids["RITEB"]

lat = PointMixin()
lat.name = "LAT"
lat.latitude = 41.54113720686182
lat.longitude = 12.918048889075166

# Function to add navaid information
def add_star(flight: Flight) -> Union[Flight, None]:
    """
    Add star information to the flight object for LIRF airport.

    Parameters
    ----------
    flight : Flight
        A flight object.

    Returns
    -------
    Flight
        A flight object with star information in an additional column.
    
    """
    
    if flight.aligned_on_navpoint(elkap, 1, "2T", "30s", 20).has():
        if flight.data["rwy"].iloc[0] in ["16L", "16R"]:
            if flight.aligned_on_navpoint(navaids["SUVOK"], 5, "2T", "10s", 20).has():
                flight.data["star"] = "ELKA2A"
                flight = hf.crop_before_wp(flight, elkap)
                return flight
    
    if flight.aligned_on_navpoint(valma, 1, "2T", "30s", 20).has():
        if flight.data["rwy"].iloc[0] in ["34L", "34R"]:
            if flight.aligned_on_navpoint(navaids["NEVUX"], 5, "2T", "10s", 20).has():
                flight.data["star"] = "VALM2C"
                flight = hf.crop_before_wp(flight, valma)
                return flight
    
    if flight.aligned_on_navpoint(riteb, 1, "2T", "30s", 20).has():
        if flight.data["rwy"].iloc[0] in ["16L", "16R"]:
            if flight.aligned_on_navpoint(navaids["EXAMA"], 5, "2T", "10s", 20).has():
                flight.data["star"] = "RITE2A"
                flight = hf.crop_before_wp(flight, riteb)
                return flight
    
    if flight.aligned_on_navpoint(lat, 1, "2T", "30s", 20).has():
        if flight.data["rwy"].iloc[0] in ["34L", "34R"]:
            if flight.aligned_on_navpoint(navaids["ODULA"], 5, "2T", "10s", 20).has():
                flight.data["star"] = "LAT2C"
                flight = hf.crop_before_wp(flight, lat)
                return flight
        
# add navaid information 
t = (
    t.iterate_lazy()
    .pipe(add_star)
    .eval(desc="add navaid", max_workers=20)
)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp.parquet"
)

### Remove GAs

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp.parquet")

# remove GAs
t = (
    t.iterate_lazy()
    .pipe(hf.remove_ga, airport)
    .eval(desc="removing GAs", max_workers=20)
)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga.parquet"
)

### Crop after runway threshold

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga.parquet")

# remove GAs
t = (
    t.iterate_lazy()
    .pipe(hf.crop_after_th, airport)
    .eval(desc="removing GAs", max_workers=20)
)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga_th.parquet"
)

### Filter

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga_th.parquet")

# Filter
t = t.filter().eval(desc="filtering", max_workers=20)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga_th_fr.parquet"
)

### Add cumulative distance

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga_th_fr.parquet")

# Filter
t = t.cumulative_distance().eval(desc="cumulative distance", max_workers=20)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_full.parquet"
)

### Generate dataframe

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_full.parquet")

# Initialize an empty list to collect data for each flight
flight_data = []

# Iterate through each flight and collect the relevant information
for flight in tqdm(t):
    id = flight.flight_id
    typecode = flight.typecode
    start = flight.start
    stop = flight.stop
    runway = flight.data.rwy.iloc[0]
    star = flight.data.star.iloc[0]
    distance = flight.data.cumdist.iloc[-1]
    
    # Append the flight data to the list as a tuple or list
    flight_data.append([id, typecode, start, stop, runway, star, distance])

# Create a DataFrame from the collected data
df = pd.DataFrame(flight_data, columns=['id','typecode','start', 'stop', 'runway', 'star', 'distance'])

# Save
df.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_df.parquet")

### Generate Subsamples for visualisation

In [14]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_full.parquet")

sample_2500 = t.sample(2500)
sample_2500.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/sample_2500.parquet")

sample_5000 = t.sample(5000)
sample_5000.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/sample_5000.parquet")

sample_10000 = t.sample(10000)
sample_10000.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/sample_10000.parquet")

elkap = t.query('star == "ELKA2A"')
elkap = elkap.sample(min(1000, len(elkap)))
elkap.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/elka2a.parquet")

valma = t.query('star == "VALM2C"')
valma = valma.sample(min(1000, len(valma)))
valma.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/valm2c.parquet")

riteb = t.query('star == "RITE2A"')
riteb = riteb.sample(min(1000, len(riteb)))
riteb.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/rite2a.parquet")

lat = t.query('star == "LAT2C"')
lat = lat.sample(min(1000, len(lat)))
lat.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/lat2c.parquet")

### Visualise

In [3]:
# load data
t = Traffic.from_file(f"/home/krum/git/STAR_shortcut_OSN_paper/data/LIRF/sample_2500.parquet")

In [ ]:
# Create a single figure for the map
fig = go.Figure()

for flight in t.sample(50):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=1.5, color="#757ef3"),
            opacity=0.5,
            name="Observed trajectories",
            showlegend=False,
        )
    )

# Add ELKAP
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["ELKAP"].latitude],
        lon=[navaids["ELKAP"].longitude],
        marker=dict(size=10, color="red"),
        text=["ELKAP"],
        textposition="top left",
        textfont=dict(color="red", size=25),
        name="ELKAP",
        showlegend=False,
    )
)

# Add VALMA navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["VALMA"].latitude],
        lon=[navaids["VALMA"].longitude],
        marker=dict(size=10, color="red"),
        text=["VALMA"],
        textposition="middle left",
        textfont=dict(color="red", size=25),
        name="VALMA",
        showlegend=False,
    )
)

# Add RITEB navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["RITEB"].latitude],
        lon=[navaids["RITEB"].longitude],
        marker=dict(size=10, color="red"),
        text=["RITEB"],
        textposition="top center",
        textfont=dict(color="red", size=25),
        name="RITEB",
        showlegend=False,
    )
)

# Add LAT navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[41.54113720686182],
        lon=[12.918048889075166],
        marker=dict(size=10, color="red"),
        text=["LAT"],
        textposition="middle right",
        textfont=dict(color="red", size=25),
        name="LAT",
        showlegend=False,
    )
)

# Update layout for the map
fig.update_layout(
    width=1000,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=7.5,
        center=dict(
            lat=41.80304294104000,
            lon=12.252616903260297,
        ),
    ),
    legend=dict(
        x=0.65,
        y=0.97,
        traceorder="normal",
        font=dict(size=25),
        bgcolor="rgba(255, 255, 255, 0.7)",
    ),
)

# Show the figure
fig.show()